In [22]:
import os
import json

REPORT_DIR_PATH = "./report_dir"

report_file_list = os.listdir(REPORT_DIR_PATH)

for report in report_file_list:
    report_path = os.getcwd() + REPORT_DIR_PATH + "/" + report
    with open(report_path, encoding="utf-8") as file:
        call_dict = dict()
        report_json = json.load(file)
        process_list = report_json["behavior"]["processes"]
        for process in process_list:
            for call in process["calls"]:  # get api call category
                if call is not None:
                    category = call["category"]
                    if category not in call_dict:
                        call_dict.update({category:1})
                    else:
                        call_dict[category] += 1
       # print(call_dict)

In [49]:
import numpy as np
from intervals import IntInterval

def hex_to_rgb(hex):
    rgb = list()
    for i in (0, 2, 4):
        decimal = int(hex[i:i+2], 16)
        rgb.append(decimal)    
    return tuple(rgb)

interval_list = list()
count_list = [0, 1, 3, 7, 12, 18, 25, 33, 42, 100, 200, float("inf")]
for i in range(len(count_list)-1):
	data_range = IntInterval.open_closed(count_list[i], count_list[i+1])  # open_closed → (,]
	interval_list.append(data_range)

# Here need to modify category!!!
# Here need to modify category!!!
# Here need to modify category!!!
color_table_str = {
    "1":    ["FFFFFF", "FFC1E0", "FFAAD5", "FF95CA", "FF79BC", "FF60AF", "FF359A", "FF0080", "F00078", "D9006C", "BF0060"],
    "2":    ["FFFFFF", "FFBFFF", "FFA6FF", "FF8EFF", "FF77FF", "FF44FF", "FF00FF", "E800E8", "D200D2", "AE00AE", "930093"],
    "3":    ["FFFFFF", "FFDAC8", "FFCBB3", "FFBD9D", "FFAD86", "FF9D6F", "FF8F59", "FF8040", "FF5809", "F75000", "D94600"],
    "4":    ["FFFFFF", "D3FF93", "CCFF80", "B7FF4A", "A8FF24", "9AFF02", "8CEA00", "82D900", "73BF00", "64A600", "548C00"],
    "5":    ["FFFFFF", "CAFFFF", "BBFFFF", "A6FFFF", "4DFFFF", "00FFFF", "00E3E3", "00CACA", "00AEAE", "009393", "005757"],
    "6":    ["FFFFFF", "C1FFE4", "ADFEDC", "96FED1", "4EFEB3", "1AFD9C", "02F78E", "02DF82", "01B468", "019858", "01814A"],
    "7":    ["FFFFFF", "D6D6AD", "CDCD9A", "C2C287", "B9B973", "AFAF61", "A5A552", "949449", "808040", "707038", "616130"],
    "8":    ["FFFFFF", "DCB5FF", "D3A4FF", "CA8EFF", "BE77FF", "B15BFF", "9F35FF", "921AFF", "8600FF", "6E00FF", "5B00AE"],
    "9":    ["FFFFFF", "FFFF6F", "FFFF37", "F9F900", "E1E100", "C4C400", "A6A600", "8C8C00", "737300", "5B5B00", "5B5B00"],
    "10":   ["FFFFFF", "FFE66F", "FFE153", "FFDC35", "FFD306", "EAC100", "D9B300", "C6A300", "AE8F00", "977C00", "796400"],
    "11":   ["FFFFFF", "D8D8EB", "C7C7E2", "B8B8DC", "A6A6D2", "9999CC", "8080C0", "7373B9", "5A5AAD", "5151A2", "484891"],
    "12":   ["FFFFFF", "97CBFF", "84C1FF", "66B3FF", "46A3FF", "2894FF", "0080FF", "0072E3", "0066CC", "005AB5", "004B97"],
    "13":   ["FFFFFF", "B9B9FF", "AAAAFF", "9393FF", "7D7DFF", "6A6AFF", "4A4AFF", "2828FF", "0000E3", "0000C6", "0000C6"],
    "14":   ["FFFFFF", "FFD1A4", "FFC78E", "FFBB77", "FFAF60", "FFA042", "FF9224", "FF8000", "EA7500", "D26900", "BB5E00"],
    "15":   ["FFFFFF", "FF9797", "FF7575", "FF5151", "FF2D2D", "FF0000", "EA0000", "CE0000", "AE0000", "930000", "750000"],
    "16":   ["FFFFFF", "93FF93", "79FF79", "53FF53", "28FF28", "00EC00", "00DB00", "00BB00", "00A600", "009100", "007500"]
}

color_table = dict()
for key, value_list in color_table_str.items():
    color_table[key] = list()
    for value in value_list:
        color_table[key].append(hex_to_rgb(value))
# print(json.dumps(color_table, indent=4))

In [ ]:
import cv2
import numpy as np

def get_x(count:int):
    for i, interval in enumerate(interval_list):
        if count in interval:
            print(i)
            return i

# temporary test function
# call_dict = {
#     "1":    1,
#     "2":    7,
#     "4":    201
# }
category_list = [*call_dict]
category_list.sort()    # make sure the order is consistent
# print(category_list)

row = 16
col = 16
img = np.zeros((row, col, 3), np.uint8)
img.fill(255)
for y, category in enumerate(category_list):    
    x = get_x(call_dict[category])
    img[y, x] = color_table[category][x]
        
# cv2.imshow('3 Channel Window', img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
# print("image shape: ", img.shape)